<a href="https://colab.research.google.com/github/Gundgisreevalli01/movie_recommendation-_system/blob/main/movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction .text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data collection and Preprocessing

In [7]:
# loading the data from the csv file  to pandas dataframe
movies_data = pd.read_csv('/content/movies.csv')

In [9]:
# printing the first five rows of the dataframe
movies_data.head()


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [10]:
# number of rows and columns in the data frame
movies_data.shape

(4803, 20)

In [11]:
#selecting the relavant features for the recommendation
selected_features =['genres','keywords','tagline','title','revenue']
print(selected_features)

['genres', 'keywords', 'tagline', 'title', 'revenue']


In [12]:
#replacing the null values with null string
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')


In [13]:
# combining all the 5 selected features
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['title']+' '+str(movies_data['revenue'])


In [14]:
print(combined_features)

0       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1       [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3       [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
                              ...                        
4798    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4799    [{"id": 35, "name": "Comedy"}, {"id": 10749, "...
4800    [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...
4801    [] [] A New Yorker in Shanghai Shanghai Callin...
4802    [{"id": 99, "name": "Documentary"}] [{"id": 15...
Length: 4803, dtype: object


In [15]:
# converting the text data to feature vector
vectorizer = TfidfVectorizer()

In [16]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [17]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 235031 stored elements and shape (4803, 21203)>
  Coords	Values
  (0, 14792)	0.4007583988456277
  (0, 6962)	0.038614178263109866
  (0, 16564)	0.41453909112984455
  (0, 10019)	0.038436338971384446
  (0, 663)	0.04458446853997582
  (0, 10068)	0.04413087250890154
  (0, 936)	0.054571916332783146
  (0, 13479)	0.053948028244031936
  (0, 9619)	0.05090970557258993
  (0, 18550)	0.050673496641959824
  (0, 13576)	0.050673496641959824
  (0, 1029)	0.10792611305602821
  (0, 12339)	0.09915627811041994
  (0, 11790)	0.10689712306005772
  (0, 7056)	0.08938105615049785
  (0, 13900)	0.07660005348124875
  (0, 7451)	0.13358668726565692
  (0, 19167)	0.30521915712037634
  (0, 20789)	0.11680670939866107
  (0, 7460)	0.12007753015176564
  (0, 11915)	0.11148387170807739
  (0, 7689)	0.12007753015176564
  (0, 19093)	0.09353096072706248
  (0, 7746)	0.10111102638621086
  (0, 20199)	0.07900821082863133
  :	:
  (4802, 5385)	0.03923526482937577
  (4802, 8396)	

Cosine similarity

In [18]:
# Getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)

In [19]:
print(similarity)

[[1.         0.3311817  0.26924952 ... 0.25711379 0.03371577 0.18664223]
 [0.3311817  1.         0.25151489 ... 0.24829746 0.03304103 0.16595386]
 [0.26924952 0.25151489 1.         ... 0.19468407 0.03945351 0.15092716]
 ...
 [0.25711379 0.24829746 0.19468407 ... 1.         0.03830166 0.18733161]
 [0.03371577 0.03304103 0.03945351 ... 0.03830166 1.         0.04346012]
 [0.18664223 0.16595386 0.15092716 ... 0.18733161 0.04346012 1.        ]]


In [20]:
print(similarity.shape)

(4803, 4803)


In [21]:
# getting the movie name from the user
movie_name = input('Enter your favourite movie name : ')

Enter your favourite movie name : bats man


In [22]:
#creating a  llist with all the movie names given in the list
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [23]:
# finding the close match for the movie  name  given by the user
find_close_match = difflib.get_close_matches(movie_name,list_of_all_titles)
print(find_close_match)

['Batman', 'Batman', 'Catwoman']


In [24]:
close_match = find_close_match[0]
print(close_match)

Batman


In [25]:
#find the index of the movie with title
index_of_the_movie = movies_data[movies_data.title == close_match].index[0]
print(index_of_the_movie)

1359


In [26]:
# getting the list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, np.float64(0.2507133703840738)), (1, np.float64(0.25311141002270404)), (2, np.float64(0.19704743600949273)), (3, np.float64(0.42273418481747965)), (4, np.float64(0.21855623262297227)), (5, np.float64(0.3274139052325664)), (6, np.float64(0.19428096276104856)), (7, np.float64(0.23986961828613534)), (8, np.float64(0.19495347506656496)), (9, np.float64(0.38786542124566004)), (10, np.float64(0.3894556920859342)), (11, np.float64(0.17427169390709546)), (12, np.float64(0.22633460034604594)), (13, np.float64(0.18103071262423256)), (14, np.float64(0.40278456663153506)), (15, np.float64(0.2833610491328394)), (16, np.float64(0.24851674960445702)), (17, np.float64(0.2374154777176752)), (18, np.float64(0.2152471471827335)), (19, np.float64(0.19560501721060294)), (20, np.float64(0.2614469838691717)), (21, np.float64(0.1425625130359843)), (22, np.float64(0.15377729971270804)), (23, np.float64(0.17107231450405366)), (24, np.float64(0.20002778210292957)), (25, np.float64(0.21081679124740046)), (26

In [27]:
len(similarity_score)

4803

In [28]:
# sorting the movies based on their smililarity score
sorted_similar_movies = sorted(similarity_score,key = lambda x:x[1],reverse = True)
print(sorted_similar_movies)

[(1359, np.float64(1.0000000000000002)), (210, np.float64(0.7266626761924111)), (428, np.float64(0.5331893763522256)), (65, np.float64(0.484397174786111)), (119, np.float64(0.4512768774057629)), (3, np.float64(0.42273418481747965)), (299, np.float64(0.4169665568128061)), (813, np.float64(0.4132378253275937)), (14, np.float64(0.40278456663153506)), (10, np.float64(0.3894556920859342)), (9, np.float64(0.38786542124566004)), (1296, np.float64(0.38443652941345646)), (30, np.float64(0.36516707351101924)), (870, np.float64(0.3651354483582571)), (41, np.float64(0.3573979429835702)), (4267, np.float64(0.35162508476559284)), (72, np.float64(0.35004569708344313)), (1512, np.float64(0.3356599045789538)), (3819, np.float64(0.33399713909012196)), (1170, np.float64(0.3328311115378833)), (1740, np.float64(0.33180817663660434)), (5, np.float64(0.3274139052325664)), (784, np.float64(0.32129270914432795)), (3854, np.float64(0.3199297657636174)), (1776, np.float64(0.3137571428876802)), (2428, np.float64(

In [29]:
# print the name of similar movies based on the index
print('movies suggested for you: \n')
i=1
for movie in   sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index == index]['title'].values[0]
  if (i<=30):
    print(i,':',title_from_index)
    i+=1

movies suggested for you: 

1 : Batman
2 : Batman & Robin
3 : Batman Returns
4 : The Dark Knight
5 : Batman Begins
6 : The Dark Knight Rises
7 : Batman Forever
8 : Superman
9 : Man of Steel
10 : Superman Returns
11 : Batman v Superman: Dawn of Justice
12 : Superman III
13 : Spider-Man 2
14 : Superman II
15 : Green Lantern
16 : Batman
17 : Suicide Squad
18 : A History of Violence
19 : Defendor
20 : The Talented Mr. Ripley
21 : Kick-Ass 2
22 : Spider-Man 3
23 : In the Name of the King: A Dungeon Siege Tale
24 : Batman: The Dark Knight Returns, Part 2
25 : The Magic Flute
26 : Brooklyn's Finest
27 : Dragonball Evolution
28 : F.I.S.T.
29 : Special
30 : The Lego Movie


MOVIE RECOMMENDATION SYSTEM

In [32]:
 movie_name = input('Enter your favourite movie name : ')
list_of_all_titles = movies_data['title'].tolist()
find_close_match = difflib.get_close_matches(movie_name,list_of_all_titles)
close_match = find_close_match[0]
index_of_the_movie = movies_data[movies_data.title == close_match].index[0]
similarity_score = list(enumerate(similarity[index_of_the_movie]))
sorted_similar_movies = sorted(similarity_score,key = lambda x:x[1],reverse = True)
print('movies suggested for you: \n')
i=1
for movie in   sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index == index]['title'].values[0]
  if (i<=30):
    print(i,':',title_from_index)
    i+=1

Enter your favourite movie name : avengers
movies suggested for you: 

1 : The Avengers
2 : Avengers: Age of Ultron
3 : Ant-Man
4 : Captain America: The Winter Soldier
5 : Captain America: Civil War
6 : Iron Man 2
7 : Thor
8 : Thor: The Dark World
9 : Captain America: The First Avenger
10 : X-Men
11 : X-Men: Apocalypse
12 : Iron Man
13 : Guardians of the Galaxy
14 : X-Men Origins: Wolverine
15 : The Incredible Hulk
16 : X2
17 : Fantastic Four
18 : Deadpool
19 : The Wolverine
20 : Iron Man 3
21 : X-Men: The Last Stand
22 : Man of Steel
23 : X-Men: First Class
24 : Kick-Ass 2
25 : Super
26 : Fantastic 4: Rise of the Silver Surfer
27 : X-Men: Days of Future Past
28 : The Amazing Spider-Man 2
29 : The Shadow
30 : Red Sonja
